# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
### <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
**Primer Examen**

**Fecha**: 14 de Marzo del 2025

**Nombre del estudiante**: Ximena Isaac Horta

**Professor**: Pablo Camarillo Ramirez

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Exam-1-ESCRIBE-AQUI-TU-NOMBRE") \
    .master("spark://34ad4bc7c499:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 15:41:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/14 15:41:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Read the Data (10 points):

Extract the employee info from the JSON column (20 points):


In [ ]:
from team_name.spark_utils import SparkUtils
employees_schema = SparkUtils.generate_schema([("employee_id", "integer"), ("employee_info", "string")])

employees_df = spark.read \
                .schema(employees_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/examen/employees.csv")

employees_df.printSchema()

employees_df.show(5, truncate=False)

root
 |-- employee_id: integer (nullable = true)
 |-- employee_info: string (nullable = true)



25/03/14 15:41:51 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/03/14 15:42:06 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/03/14 15:42:21 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/03/14 15:42:36 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/03/14 15:42:51 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/03/14 15:43:06 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

In [ ]:
from team_name.spark_utils import SparkUtils
departments_schema = SparkUtils.generate_schema([("department_id", "integer"), ("department_name", "string"), ("location", "string")])

departments_df = spark.read \
                .schema(departments_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/examen/departments.csv")

departments_df.printSchema()

departments_df.show(5, truncate=False)

In [ ]:
from pyspark.sql.functions import get_json_object

employees_df = employees_df.withColumn("name", get_json_object(employees_df.employee_info, "$.name")) \
                            .withColumn("department_id", get_json_object(employees_df.employee_info, '$.department_id')) \
                            .withColumn("salary", get_json_object(employees_df.employee_info, '$.salary')) \
                            .withColumn("hire_date", get_json_object(employees_df.employee_info, '$.hire_date')) \
                            .drop("employee_info")

employees_df.show(truncate=False)

+-----------+---------+-------------+---------+----------+
|employee_id|name     |department_id|salary   |hire_date |
+-----------+---------+-------------+---------+----------+
|1          |Caitlyn  |103          |115959.78|2002-06-10|
|2          |Rachel   |104          |100820.16|2009-07-01|
|3          |Carrie   |105          |114421.44|1998-12-10|
|4          |Renee    |104          |54688.13 |1995-03-17|
|5          |Gabriella|109          |106267.03|1995-02-09|
|6          |Kristen  |101          |88237.54 |2010-11-15|
|7          |Jonathan |102          |39323.42 |2012-06-30|
|8          |Michelle |101          |64262.85 |2005-10-30|
|9          |Michelle |105          |103521.88|1991-07-10|
|10         |Lisa     |110          |55435.93 |2016-03-25|
|11         |Cheryl   |103          |88073.75 |2020-08-21|
|12         |Mikayla  |107          |95192.05 |2022-05-13|
|13         |Lisa     |104          |36032.49 |2019-05-16|
|14         |Sean     |108          |64904.69 |2021-06-0

Join Operations (10 points):

In [74]:
joinData_df = employees_df.join(departments_df, employees_df.department_id == departments_df.department_id, "inner")
joinData_df.show(5, truncate=False)


+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+
|employee_id|name     |department_id|salary   |hire_date |department_id|department_name    |location     |
+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+
|1          |Caitlyn  |103          |115959.78|2002-06-10|103          |Sales and Marketing|Chicago      |
|2          |Rachel   |104          |100820.16|2009-07-01|104          |Data Engineering   |Zapopan      |
|3          |Carrie   |105          |114421.44|1998-12-10|105          |Data Science       |Seattle      |
|4          |Renee    |104          |54688.13 |1995-03-17|104          |Data Engineering   |Zapopan      |
|5          |Gabriella|109          |106267.03|1995-02-09|109          |Customer Service   |San Francisco|
+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+
only showing top 5 rows



Transformations (Using when()) (10 points):

In [76]:
from pyspark.sql.functions import when

joinData_df = joinData_df.withColumn('salary_category', when(joinData_df['salary'] >= 55000, "High").otherwise("Low"))

joinData_df.show(n=5)

+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+
|employee_id|     name|department_id|   salary| hire_date|department_id|    department_name|     location|salary_category|
+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+
|          1|  Caitlyn|          103|115959.78|2002-06-10|          103|Sales and Marketing|      Chicago|           High|
|          2|   Rachel|          104|100820.16|2009-07-01|          104|   Data Engineering|      Zapopan|           High|
|          3|   Carrie|          105|114421.44|1998-12-10|          105|       Data Science|      Seattle|           High|
|          4|    Renee|          104| 54688.13|1995-03-17|          104|   Data Engineering|      Zapopan|            Low|
|          5|Gabriella|          109|106267.03|1995-02-09|          109|   Customer Service|San Francisco|           High|
+-----------+---

Filter and Group (20 points):

In [77]:
high_salary_df = joinData_df.filter(joinData_df["salary_category"] == "High")
low_salary_df = joinData_df.filter(joinData_df["salary_category"] == "Low")

high_salary_df.show(n=5)
low_salary_df.show(n=5)

+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+
|employee_id|     name|department_id|   salary| hire_date|department_id|    department_name|     location|salary_category|
+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+
|          1|  Caitlyn|          103|115959.78|2002-06-10|          103|Sales and Marketing|      Chicago|           High|
|          2|   Rachel|          104|100820.16|2009-07-01|          104|   Data Engineering|      Zapopan|           High|
|          3|   Carrie|          105|114421.44|1998-12-10|          105|       Data Science|      Seattle|           High|
|          5|Gabriella|          109|106267.03|1995-02-09|          109|   Customer Service|San Francisco|           High|
|          6|  Kristen|          101| 88237.54|2010-11-15|          101|    Human Resources|    San Diego|           High|
+-----------+---

In [80]:
from pyspark.sql.functions import col

high_salary_df = high_salary_df.withColumn("salary", col("salary").cast("double"))
low_salary_df = low_salary_df.withColumn("salary", col("salary").cast("double"))


avg_high_salary_df = high_salary_df.groupBy("department_name").avg("salary")
avg_low_salary_df = low_salary_df.groupBy("department_name").avg("salary")

avg_high_salary_df.show(n=5)
avg_low_salary_df.show(n=5)


+--------------------+------------------+
|     department_name|       avg(salary)|
+--------------------+------------------+
|Corporate Strateg...|102741.38324414717|
| Sales and Marketing|100839.65275449108|
|    Data Engineering|101626.29492163012|
|Research and Deve...|  98714.3003086419|
|Finance and Accou...|100731.07877887784|
+--------------------+------------------+
only showing top 5 rows

+--------------------+------------------+
|     department_name|       avg(salary)|
+--------------------+------------------+
|Corporate Strateg...|41590.741833333326|
| Sales and Marketing| 41150.40277777778|
|    Data Engineering| 41358.50794117647|
|Finance and Accou...|42740.952888888874|
|Research and Deve...| 41426.43521126761|
+--------------------+------------------+
only showing top 5 rows



Sort (10 points): 


In [81]:
top_5_high_salary_df = high_salary_df.orderBy(col("salary").desc()).limit(5)
top_5_high_salary_df.select("name", "salary").show(5)

+---------+---------+
|     name|   salary|
+---------+---------+
|Gabriella|149989.73|
|Katherine| 149979.3|
|     Ryan| 149963.1|
|  Caitlyn|149956.54|
|     Mark|149915.56|
+---------+---------+



In [82]:
top_5_low_salary_df = low_salary_df.orderBy(col("salary").desc()).limit(5)
top_5_low_salary_df.select("name", "salary").show(5)

+-----+--------+
| name|  salary|
+-----+--------+
|Linda|54993.53|
|Tammy|54991.71|
|Aaron|54989.45|
|Craig| 54945.2|
|Aaron| 54937.3|
+-----+--------+



Find the number of employees with more years in the company (15 points).

In [92]:
from pyspark.sql.functions import year, col
calculo_df = joinData_df.withColumn("years_in_company", 2025 - year(col("hire_date")))
max_years_in_company = calculo_df.agg({"years_in_company": "max"}).collect()[0][0]
employees_with_max_years = calculo_df.filter(col("years_in_company") == max_years_in_company)
employees_with_max_years.select("name", "years_in_company").orderBy(col("years_in_company"), ascending=False).show()
num_employees_with_max_years = employees_with_max_years.count()



+--------+----------------+
|    name|years_in_company|
+--------+----------------+
|     Ana|              37|
|    Seth|              37|
|   Megan|              37|
|   Sarah|              37|
|    Mark|              37|
|    Luke|              37|
|  Carrie|              37|
|   James|              37|
|  Brandy|              37|
|Jonathan|              37|
|   Jorge|              37|
| William|              37|
| Tristan|              37|
|   Tammy|              37|
| Raymond|              37|
|  Rachel|              37|
| Tristan|              37|
|  Cheryl|              37|
|     Amy|              37|
|Jennifer|              37|
+--------+----------------+
only showing top 20 rows



In [ ]:
sc.stop()

Display the  Spark Plan (5 points):